## IMPORTS

In [5]:
import warnings
#warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import time

In [6]:
%matplotlib inline

In [7]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.0 MB/s eta 0:00:00


In [8]:
from keras.layers import Input, LSTM, Dense, concatenate
from keras.models import Model
from transformers import BertTokenizer, TFBertModel
from keras.layers import Input, Dense
from keras.models import Model

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

MAX_LENGTH = 512 #can be changed bruh

input_ids = Input(shape=(MAX_LENGTH,), dtype='int32', name='input_ids')
attention_mask = Input(shape=(MAX_LENGTH,), dtype='int32', name='attention_mask')

bert_output = bert_model([input_ids, attention_mask])
# Here, we're using the [CLS] token's representation as the sequence representation
sequence_output = bert_output.last_hidden_state[:, 0, :]

# Add the classification layer
sentiment_output = Dense(1, activation='sigmoid')(sequence_output)  # Binary classification

sentiment_model = Model(inputs=[input_ids, attention_mask], outputs=sentiment_output)
sentiment_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

## Calendar Preprocess

In [58]:
import os
import glob
from datetime import datetime
from pandas.core.frame import DataFrame
from numpy.core.multiarray import datetime_as_string
import pandas as pd
from dateutil.rrule import rrule, WEEKLY, MO, TU, WE, TH, FR, SA, SU
from datetime import datetime, date


WEEKDAYS_MAP = {
    "MO": MO,
    "TU": TU,
    "WE": WE,
    "TH": TH,
    "FR": FR,
    "SA": SA,
    "SU": SU
}

def generate_recurring_events(row, last_event_date):
    # Extract repeating patterns from the Notes column
    if 'Notes' not in row or pd.isnull(row['Notes']):
      return []

    rules = row['Notes'].split(";")
    freq_pattern = [rule for rule in rules if "FREQ=" in rule][0].split("=")[1]

    until_date = None
    if "UNTIL=" in row['Notes']:
        until_pattern = [rule for rule in rules if "UNTIL=" in rule][0].split("=")[1]
        until_date = datetime.strptime(until_pattern[:8], '%Y%m%d').date()

    if until_date and until_date < last_event_date:
        last_event_date = until_date

    byday_pattern = [rule for rule in rules if "BYDAY=" in rule]
    if byday_pattern:
        days = byday_pattern[0].split("=")[1].split(',')
    else:
        days = []

    start_date = pd.to_datetime(row['start_date']).date()

    recurring_dates = list(rrule(freq=WEEKLY, dtstart=start_date, until=last_event_date, byweekday=[WEEKDAYS_MAP[day] for day in days]))
    return [{'start_date': date,
             'date_column_start': date,
             'duration': row['duration'],
             'Title': row['Title'],
             'repeating': 1} for date in recurring_dates]


def preprocessCals():
  # List all CSV files in the "calendars" directory
  csv_files = glob.glob("calendars/*.csv")
  # Read each CSV file into a DataFrame and store them in a list
  dfs = [pd.read_csv(file) for file in csv_files]
  # Concatenate all the DataFrames
  calendar_df = pd.concat(dfs, ignore_index=True)
  calendar_df = calendar_df.rename(columns={"Given planned earliest start": "start", "Given planned earliest end": "end"})
  #From the csv file, create another calendar_dfframe that parses events per day. and generates features
  calendar_df[['start_date', 'start_time']] = calendar_df['start'].str.split(' ', 1, expand=True)
  calendar_df['start_time'] = calendar_df['start_time'].str.slice(0,5)
  calendar_df['start_time'] = calendar_df['start_time'].astype(str)
  calendar_df[['end_date', 'end_time']] = calendar_df['end'].str.split(' ', 1, expand=True)
  calendar_df['end_time'] = calendar_df['end_time'].str.slice(0,5)
  calendar_df['end_time'] = calendar_df['end_time'].astype(str)
  # Function to convert time strings to total minutes
  def convert_to_minutes(time_str):
      if(time_str == 'nan'):
        return 540
      hours, minutes = map(int, time_str.split(':'))
      return hours * 60 + minutes

  # Apply the function to the column
  calendar_df['total_minutes_start'] = calendar_df['start_time'].apply(convert_to_minutes)
  calendar_df['total_minutes_end'] = calendar_df['end_time'].apply(convert_to_minutes)

  # Calculate duration taking into account time boundaries
  calendar_df['duration'] = calendar_df.apply(lambda row: row['total_minutes_end'] - row['total_minutes_start'] if row['total_minutes_end'] >= row['total_minutes_start'] else (24*60 - row['total_minutes_start']) + row['total_minutes_end'], axis=1)
  calendar_df = calendar_df.drop(columns=['start_time', 'end_time'])

  #add a repeat column for repeating events
  calendar_df['repeating'] = calendar_df['Notes'].str.slice(0,4)
  calendar_df['repeating'] = calendar_df['repeating'].apply(lambda x: 1 if x == 'FREQ' else 0)


  calendar_df['date_column_start'] = pd.to_datetime(calendar_df['start_date'], format='%m/%d/%Y')
  calendar_df['date_column_end'] = pd.to_datetime(calendar_df['end_date'], format='%m/%d/%Y')

  # Sort the calendar_dfFrame based on the date_column
  calendar_df = calendar_df.sort_values(by='date_column_start')

  # Get the last event's date
  last_event_date = calendar_df['date_column_end'].max().date()

  # For each row, generate recurring events
  new_rows = []
  for _, row in calendar_df.iterrows():
      if row['repeating'] == 1:
          new_events = generate_recurring_events(row, last_event_date)
          new_rows.extend(new_events)

  print(new_rows)

  # Create a DataFrame from the new rows
  new_df = pd.DataFrame(new_rows)
  calendar_df = pd.concat([calendar_df, new_df], ignore_index=True)

  #calendar_df['date_column_start'] = pd.to_datetime(calendar_df['start_date'], format='%m/%d/%Y')

  try:
    calendar_df = calendar_df.drop(columns=['Notes', 'Assigned Resources', 'Additional Title', 'start', 'end'])
  except:
    print("error dropping")

  # Sort by ascending date again
  calendar_df = calendar_df.sort_values(by='date_column_start')
  calendar_df = calendar_df.reset_index(drop=True)

  # Reset the index if needed
  calendar_df = calendar_df.reset_index(drop=True)
  #drop last row as it does not contain any useful calendar_df
  calendar_df = calendar_df.drop([len(calendar_df)-1])
  # Remove rows where all data is missing or NaN
  calendar_df = calendar_df.dropna(how='all')
  calendar_df = calendar_df.dropna(subset=['Title', 'date_column_start', 'repeating', 'duration'])

  return calendar_df

calendar_df = preprocessCals()

[{'start_date': datetime.datetime(2022, 9, 2, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 2, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 3, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 3, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 4, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 4, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 5, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 5, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 6, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 6, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 7, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 7, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 

<ipython-input-58-184d36917397>:62: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  calendar_df[['start_date', 'start_time']] = calendar_df['start'].str.split(' ', 1, expand=True)
<ipython-input-58-184d36917397>:65: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  calendar_df[['end_date', 'end_time']] = calendar_df['end'].str.split(' ', 1, expand=True)


## Spending Preprocess

In [35]:
from pandas.core.frame import DataFrame
from numpy.core.multiarray import datetime_as_string

def preprocessSpend():
  #NOTE: in final code replace the csv file with the input from the website
  spending_df = pd.read_csv("Spendings.csv")

  # Convert the date_column to datetime objects
  spending_df['date_column_start'] = pd.to_datetime(spending_df['Date'], format='%m/%d/%Y')

  # Remove rows where all data is missing or NaN
  spending_df = spending_df.dropna(how='all')

  # Sort the DataFrame based on the date_column
  spending_df = spending_df.sort_values(by='date_column_start')

  # Reset the index if needed
  spending_df = spending_df.reset_index(drop=True)

  # Drop the last row if it does not contain any useful data
  spending_df = spending_df.drop([len(spending_df)-1])
  return spending_df


## MODEL

In [65]:
from keras.layers import Bidirectional, Dropout

# 7 day density (past) a frequency array of the past 7 days
event_density_past_input = Input(shape=(7, 1), name='event_density_past_input')
past_density_event = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(event_density_past_input)
past_density_event = LSTM(32)(past_density_event)


# 7 day density (future) a frequency array of the future 7 days
event_density_future_input = Input(shape=(7, 1), name='event_density_future_input')
future_density_event = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(event_density_future_input)
future_density_event = LSTM(32)(future_density_event)

# BERT embeddings input for past 10 events
bert_past_event_input = Input(shape=(10, 768), name='bert_past_event_input')
x1_past_event = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(bert_past_event_input)
x1_past_event = LSTM(64)(x1_past_event)

# Structured data input (dates, durations, repeats for past 10 events)
struct_past_event_input = Input(shape=(10, 4), name='struct_past_event_input')  # Previously (10, 3)
x2_past_event = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(struct_past_event_input)
x2_past_event = LSTM(32)(x2_past_event)

# Past spending vendor BERT
bert_past_spending_input = Input(shape=(10, 768), name='bert_past_spending_input')
x4_past_spending = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(bert_past_spending_input)
x4_past_spending = LSTM(64)(x4_past_spending)

# 7 day density spending a frequency array of the spending events of the past 7 days
spending_density_input = Input(shape=(7, 1), name='spending_density_input')
spending_density = LSTM(64, return_sequences=True)(spending_density_input)
spending_density = LSTM(32)(spending_density)

# Past spending data: Date, Amount
struct_past_spending_input = Input(shape=(10, 2), name='struct_past_spending_input')  # I assume this should be (10, 2) and not (10, 3) as described
x5_past_spending = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(struct_past_spending_input)
x5_past_spending = LSTM(32)(x5_past_spending)

# BERT embeddings input for next 10 events
bert_future_input = Input(shape=(10, 768), name='bert_future_input')
x1_future = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(bert_future_input)
x1_future = LSTM(64)(x1_future)

# Structured data input (dates, durations, repeats for next 10 events)
struct_future_input = Input(shape=(10, 4), name='struct_future_input')  # Previously (10, 3)
x2_future = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(struct_future_input)
x2_future = LSTM(32)(x2_future)

# Current date input (assuming encoded as a single value)
current_date_input = Input(shape=(1,), name='current_date_input')

# Merging all processed inputs
merged = concatenate([x1_past_event, x2_past_event, x4_past_spending, x5_past_spending, x1_future, x2_future, past_density_event, future_density_event, spending_density, current_date_input])


# Continue processing the merged data
hidden_layer_1 = Dense(64, activation='relu')(merged)
dropout_1 = Dropout(0.5)(hidden_layer_1)
hidden_layer_2 = Dense(32, activation='relu')(dropout_1)
dropout_2 = Dropout(0.5)(hidden_layer_2)
hidden_layer_3 = Dense(16, activation='relu')(dropout_2)
hidden_layer_4 = Dense(16, activation='relu')(hidden_layer_3)
output = Dense(7, activation='relu')(hidden_layer_4)

from keras import backend as K

def custom_mse(y_true, y_pred):
    # Mean squared error for the individual outputs
    individual_mse = K.mean(K.square(y_pred - y_true), axis=-1)

    # Mean squared error for the sum of outputs
    sum_mse = K.square(K.sum(y_pred, axis=-1) - K.sum(y_true, axis=-1))

    # Combine the individual and sum MSE
    return individual_mse + sum_mse

model = Model(inputs=[bert_past_event_input, struct_past_event_input, bert_past_spending_input, struct_past_spending_input, bert_future_input, struct_future_input, event_density_past_input, event_density_future_input, spending_density_input, current_date_input], outputs=output)
model.compile(optimizer='adam', loss=custom_mse)

In [66]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

MAX_TOKENS = 768


def pad_or_truncate(tokens):
    if len(tokens) < MAX_TOKENS:
        return tokens + [0] * (MAX_TOKENS - len(tokens))
    return tokens[:MAX_TOKENS]

def preprocess_data(calendar_df, spending_df, tokenizer):
    calendar_df['event_tokens'] = calendar_df['Title'].apply(lambda x: pad_or_truncate(tokenizer.encode(x, add_special_tokens=True, truncation=True)))
    spending_df['vendor_tokens'] = spending_df['Name'].apply(lambda x: pad_or_truncate(tokenizer.encode(x, add_special_tokens=True, truncation=True)))
    calendar_df['day_of_week'] = calendar_df['date_column_start'].dt.dayofweek  # This will give you numbers 0-6 for Monday-Sunday.
    return calendar_df, spending_df

def get_date_range(calendar_df):
    start_date = calendar_df['date_column_start'].iloc[9]  # 10th event
    end_date = calendar_df['date_column_start'].iloc[-1]
    return pd.date_range(start=start_date, end=end_date)



# Helper function to get the past and future 10 events/spendings and spending for next week
def get_data(date):

    # Check if 'Amount' is of type string, if not, convert it
    if spending_df['Amount'].dtype != 'object':
        spending_df['Amount'] = spending_df['Amount'].astype(str)

    # Convert 'Amount' column to float
    spending_df['Amount'] = spending_df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)

    past_events = calendar_df[calendar_df['date_column_start'] < date].tail(10)
    future_events = calendar_df[calendar_df['date_column_start'] > date].head(10)
    past_spendings = spending_df[spending_df['date_column_start'] < date].tail(10)

    # Now sum up 'Amount' after conversion
    #next_week_spending = spending_df[(spending_df['date_column_start'] > date) & (spending_df['date_column_start'] <= date + timedelta(days=7))].sum()['Amount']

    # Initialize a list for daily spending
    daily_spending = []

     # Iterate over the next 7 days
    for i in range(7):
        # Calculate the start and end dates for each day
        start_date = date + timedelta(days=i)
        end_date = date + timedelta(days=i + 1)

        # Filter the spending data for the current day
        day_spending = spending_df[(spending_df['date_column_start'] > start_date) & (spending_df['date_column_start'] <= end_date)]['Amount']

        # Calculate the total spending for the day and append it to the list
        total_day_spending = day_spending.sum()
        daily_spending.append(total_day_spending)

    # Convert the list to a NumPy array for further processing
    next_week_spending = daily_spending


    # Calculate past week event density
    past_week_event_density = []
    for i in range(7):
        day_events = calendar_df[calendar_df['date_column_start'] == date - timedelta(days=i+1)]
        past_week_event_density.append(len(day_events))
    past_week_event_density = np.array(past_week_event_density[::-1]).reshape(7,1) # Reverse to have oldest day first

    # Calculate future week event density
    future_week_event_density = []
    for i in range(7):
        day_events = calendar_df[calendar_df['date_column_start'] == date + timedelta(days=i+1)]
        future_week_event_density.append(len(day_events))
    future_week_event_density = np.array(future_week_event_density).reshape(7,1)

    # Calculate past week spending density
    past_week_spending_density = []
    for i in range(7):
        day_spending = spending_df[spending_df['date_column_start'] == date - timedelta(days=i+1)]
        total_spent = day_spending['Amount'].sum()
        past_week_spending_density.append(total_spent)
    past_week_spending_density = np.array(past_week_spending_density[::-1]).reshape(7,1) # Reverse to have oldest day first

    return past_events, future_events, past_spendings, next_week_spending, past_week_event_density, future_week_event_density, past_week_spending_density



def prepare_training_data(calendar_df, spending_df, tokenizer):
    date_range = get_date_range(calendar_df)
    X_train_past_event_tokens = []
    X_train_past_event_structured = []
    X_train_past_spending_tokens = []
    X_train_past_spending_structured = []
    X_train_future_event_tokens = []
    X_train_future_event_structured = []
    X_train_current_date_encoded = []
    X_train_past_week_event_density = []
    X_train_future_week_event_density = []
    X_train_past_week_spending_density = []
    y_train = []
    for date in date_range:# Prepare the training data
      past_events, future_events, past_spendings, next_week_spending, past_week_event_density, future_week_event_density, past_week_spending_density = get_data(date)


      # If there are less than 10 future events, fill up with default events
      dayINC = 1
      while len(future_events) < 10:
          default_event = pd.DataFrame({
              'Title': ['Default_Event'],
              'date_column_start': [date + timedelta(dayINC)],  # default date set to 1 year in the future, can be adjusted
              'repeating': [0],
              'duration': [0],
              'day_of_week': [(date + timedelta(days=dayINC)).weekday()], # Added day_of_week for default event,
              'event_tokens': [pad_or_truncate(tokenizer.encode('Default_Event', add_special_tokens=True, truncation=True))]
          })
          future_events = pd.concat([future_events, default_event], ignore_index=True)
          dayINC += 1

      if len(past_events) >= 10 and len(future_events) >= 10:

        past_events['date_column_start'] = past_events['date_column_start'].apply(lambda x: x.timestamp())
        future_events['date_column_start'] = future_events['date_column_start'].apply(lambda x: x.timestamp())
        past_spendings['date_column_start'] = past_spendings['date_column_start'].apply(lambda x: x.timestamp())

        # Append BERT tokens for past and future events and past spending
        X_train_past_event_tokens.append(np.array(past_events['event_tokens'].tolist()))
        X_train_past_spending_tokens.append(np.array(past_spendings['vendor_tokens'].tolist()))
        X_train_future_event_tokens.append(np.array(future_events['event_tokens'].tolist()))

        # Append structured data for past and future events and past spending
        X_train_past_event_structured.append(np.array(past_events[['date_column_start', 'repeating', 'duration', 'day_of_week']]))
        X_train_past_spending_structured.append(np.array(past_spendings[['date_column_start', 'Amount']]))
        X_train_future_event_structured.append(np.array(future_events[['date_column_start', 'repeating', 'duration', 'day_of_week']]))

        # Add density data to training lists
        X_train_past_week_event_density.append(past_week_event_density)
        X_train_future_week_event_density.append(future_week_event_density)
        X_train_past_week_spending_density.append(past_week_spending_density)

        # Append the current date (encoded as a single value)
        X_train_current_date_encoded.append([date.timestamp()])
        y_train.append(next_week_spending)

    # Convert lists to arrays
    X_train_past_event_tokens = np.array(X_train_past_event_tokens)
    X_train_past_event_structured = np.array(X_train_past_event_structured)
    X_train_past_spending_tokens = np.array(X_train_past_spending_tokens)
    X_train_past_spending_structured = np.array(X_train_past_spending_structured)
    X_train_past_week_event_density = np.array(X_train_past_week_event_density)
    X_train_future_week_event_density = np.array(X_train_future_week_event_density)
    X_train_past_week_spending_density = np.array(X_train_past_week_spending_density)

    X_train_future_event_tokens = np.array(X_train_future_event_tokens)
    X_train_future_event_structured = np.array(X_train_future_event_structured)
    X_train_current_date_encoded = np.array(X_train_current_date_encoded)
    y_train = np.array(y_train)


    model_inputs = [X_train_past_event_tokens,
                X_train_past_event_structured,
                X_train_past_spending_tokens,
                X_train_past_spending_structured,
                X_train_future_event_tokens,
                X_train_future_event_structured,
                X_train_past_week_event_density,
                X_train_future_week_event_density,
                X_train_past_week_spending_density,
                X_train_current_date_encoded]

    return (model_inputs, y_train)

calendar_df = preprocessCals()
spending_df = preprocessSpend()
calendar_df, spending_df = preprocess_data(calendar_df, spending_df, tokenizer)
model_inputs, y_train = prepare_training_data(calendar_df, spending_df, tokenizer)

<ipython-input-58-184d36917397>:62: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  calendar_df[['start_date', 'start_time']] = calendar_df['start'].str.split(' ', 1, expand=True)
<ipython-input-58-184d36917397>:65: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  calendar_df[['end_date', 'end_time']] = calendar_df['end'].str.split(' ', 1, expand=True)
<ipython-input-66-5835069575e5>:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  spending_df['Amount'] = spending_df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)
<ipython-input-66-5835069575e5>:34: FutureWarning: The default value of regex will change from True to False in a future version

[{'start_date': datetime.datetime(2022, 9, 2, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 2, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 3, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 3, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 4, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 4, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 5, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 5, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 6, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 6, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 7, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 7, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 

<ipython-input-66-5835069575e5>:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  spending_df['Amount'] = spending_df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)
<ipython-input-66-5835069575e5>:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  spending_df['Amount'] = spending_df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)
<ipython-input-66-5835069575e5>:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  spending_df['Amount'] = spending_df['Amount'].str.replace('$', '').str.repla

In [67]:
# Train the model
model.fit(model_inputs, y_train, epochs=30, batch_size=32)

Epoch 1/30
13/13 [==============================] - 98s 916ms/step - loss: 20571269850202112.0000
Epoch 2/30
13/13 [==============================] - 10s 753ms/step - loss: 4222114986983424.0000
Epoch 3/30
13/13 [==============================] - 14s 1s/step - loss: 1144764476948480.0000
Epoch 4/30
13/13 [==============================] - 13s 1s/step - loss: 525375935348736.0000
Epoch 5/30
13/13 [==============================] - 13s 1s/step - loss: 320293696438272.0000
Epoch 6/30
13/13 [==============================] - 14s 1s/step - loss: 304743331135488.0000
Epoch 7/30
13/13 [==============================] - 14s 1s/step - loss: 217428608942080.0000
Epoch 8/30
13/13 [==============================] - 14s 1s/step - loss: 151084584665088.0000
Epoch 9/30
13/13 [==============================] - 14s 1s/step - loss: 133072238411776.0000
Epoch 10/30
13/13 [==============================] - 15s 1s/step - loss: 52785080958976.0000
Epoch 11/30
13/13 [==============================] - 14s 1s/

In [68]:
def prepare_model_inputs(calendar_df, get_data, tokenizer):
    # Now, only generate data for the second-to-last event date
    date = calendar_df['date_column_start'].iloc[-601]

    # Fetch data for that specific date
    past_events, future_events, past_spendings, next_week_spending, past_week_event_density, future_week_event_density, past_week_spending_density = get_data(date)

    dayINC = 1
    while len(future_events) < 10:
        default_event = pd.DataFrame({
            'Title': ['Default_Event'],
            'date_column_start': [date + timedelta(dayINC)],  # default date set to 1 year in the future, can be adjusted
            'repeating': [0],
            'duration': [0],
            'day_of_week': [(date + timedelta(days=dayINC)).weekday()], # Added day_of_week for default event,
            'event_tokens': [pad_or_truncate(tokenizer.encode('Default_Event', add_special_tokens=True, truncation=True))]
        })
        future_events = pd.concat([future_events, default_event], ignore_index=True)
        dayINC += 1

    if len(past_events) >= 10 and len(future_events) >= 10:
        past_events['date_column_start'] = past_events['date_column_start'].apply(lambda x: x.timestamp())
        future_events['date_column_start'] = future_events['date_column_start'].apply(lambda x: x.timestamp())
        past_spendings['date_column_start'] = past_spendings['date_column_start'].apply(lambda x: x.timestamp())

        X_TEST_past_event_tokens = np.array(past_events['event_tokens'].tolist())
        X_TEST_past_spending_tokens = np.array(past_spendings['vendor_tokens'].tolist())
        X_TEST_future_event_tokens = np.array(future_events['event_tokens'].tolist())

        X_TEST_past_event_structured = np.array(past_events[['date_column_start', 'repeating', 'duration', 'day_of_week']])
        X_TEST_past_spending_structured = np.array(past_spendings[['date_column_start', 'Amount']])
        X_TEST_future_event_structured = np.array(future_events[['date_column_start', 'repeating', 'duration', 'day_of_week']])

        X_TEST_past_week_event_density = past_week_event_density
        X_TEST_future_week_event_density = future_week_event_density
        X_TEST_past_week_spending_density = past_week_spending_density

        X_TEST_current_date_encoded = np.array([date.timestamp()])
        Y_TEST = next_week_spending

    model_inputs = [X_TEST_past_event_tokens,
                    X_TEST_past_event_structured,
                    X_TEST_past_spending_tokens,
                    X_TEST_past_spending_structured,
                    X_TEST_future_event_tokens,
                    X_TEST_future_event_structured,
                    X_TEST_past_week_event_density,
                    X_TEST_future_week_event_density,
                    X_TEST_past_week_spending_density,
                    X_TEST_current_date_encoded]

    model_inputs = [np.expand_dims(x, axis=0) for x in model_inputs]

    return model_inputs, Y_TEST


In [69]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [70]:
#from google.colab import files
#files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
from keras.models import load_model
loaded_model = load_model('model.h5', custom_objects={'custom_mse': custom_mse})
calendar_df = preprocessCals()
spending_df = preprocessSpend()
preprocess_data(calendar_df, spending_df, tokenizer)
model_inputs, Y_TEST = prepare_model_inputs(calendar_df, get_data, tokenizer)
predicted_spending = loaded_model.predict(model_inputs)
print(Y_TEST)
print(predicted_spending)

<ipython-input-58-184d36917397>:62: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  calendar_df[['start_date', 'start_time']] = calendar_df['start'].str.split(' ', 1, expand=True)
<ipython-input-58-184d36917397>:65: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  calendar_df[['end_date', 'end_time']] = calendar_df['end'].str.split(' ', 1, expand=True)
<ipython-input-66-5835069575e5>:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  spending_df['Amount'] = spending_df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)


[{'start_date': datetime.datetime(2022, 9, 2, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 2, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 3, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 3, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 4, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 4, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 5, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 5, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 6, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 6, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 9, 7, 0, 0), 'date_column_start': datetime.datetime(2022, 9, 7, 0, 0), 'duration': 120, 'Title': 'Bjj', 'repeating': 1}, {'start_date': datetime.datetime(2022, 